<a href="https://colab.research.google.com/github/jumbokh/ML-Class/blob/main/yenlung/%E4%BD%9C%E6%A5%AD%E5%9B%9B_%E8%AC%9D%E5%9D%A4%E9%81%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
import tensorflow as tf  # Basic sources
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

## 掛載我的雲端硬碟

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -fs /content/gdrive/My\ Drive /app
%cd /app/data

Mounted at /content/gdrive
/content/gdrive/My Drive/data


## 在虛擬機上安裝、使用預先下載的 cuDNN 驅動程式

In [6]:
!tar -xzvf /app/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# 檢查是否安裝成功
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


### 1. 讀入 MNSIT 數據集

In [7]:
from tensorflow.keras.datasets import mnist

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [12]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255. # Normalization 使用浮點數

In [13]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255. # Normalization 使用浮點數

In [14]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 檢查訓練集和測試集的筆數及維度

In [15]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


### 3. step 1: 打造函數學習機 (CNN)

In [16]:
model = Sequential()

In [17]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [18]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [19]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [20]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [21]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [22]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [23]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [24]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [26]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [27]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(54, activation='relu'))

In [33]:
model.add(Dropout(0.2))   # 砍掉 20% 資料

In [34]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

# 網路修改如下:
## 1. 在原來的基礎上，每個卷積層皆做兩次，
## 2. optimizer 改用 Adam
## 3. 在輸出前砍掉 20%資料

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 32)          9

#### 組裝

In [37]:
model.compile(loss='mse', optimizer=Adam(lr=0.001),
             metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### 4. step 2: fit

In [38]:
model.fit(x_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
469/469 [==============================] - 36s 12ms/step - loss: 0.0172 - accuracy: 0.8750
Epoch 2/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0044 - accuracy: 0.9722
Epoch 3/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0033 - accuracy: 0.9795
Epoch 4/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0025 - accuracy: 0.9841
Epoch 5/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0022 - accuracy: 0.9862
Epoch 6/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0019 - accuracy: 0.9880
Epoch 7/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0018 - accuracy: 0.9890
Epoch 8/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0016 - accuracy: 0.9899
Epoch 9/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0015 - accuracy: 0.9905
Epoch 10/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0013 - accur

## 在第一個 Epoch 就看到明顯的效果，
## 36s 12ms/step - loss: 0.0172 - accuracy: 0.8750
## 使用 cuDNN driver+GPU 效能明顯增加
### 原始:
* Epoch 1/12
* 469/469 [==============================] - 45s 93ms/step - loss: 0.0895 - accuracy: 0.1921
* Epoch 2/12
* 469/469 [==============================] - 44s 93ms/step - loss: 0.0879 - accuracy: 0.3694
### 改善後:
* Epoch 1/15
* 469/469 [==============================] - 36s 12ms/step - loss: 0.0172 - accuracy: 0.8750
* Epoch 2/15
* 469/469 [==============================] - 5s 12ms/step - loss: 0.0044 - accuracy: 0.9722
## 原來的結果如下, 
loss: 0.0065 - accuracy: 0.9593
## 修改及增加Layer實驗結果:
loss: 0.0011 - accuracy: 0.9933

### Step 3. 預測

## tensorflow 2.5 之後取消 predict_classes
## 改寫如下

16

This function were removed in TensorFlow version 2.6. 
According to the keras in [rstudio reference](https://keras.rstudio.com/reference/predict_proba.html#details)

In [40]:
result = (model.predict(x_test) > 0.5).astype("int32")
result

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

### 看看測試資料表現如何

In [41]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0018 - accuracy: 0.9890


In [43]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 98.90%


In [44]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [45]:
from ipywidgets import interact_manual

In [46]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [48]:
#model.save('/app/data/myCNNmodel.h5')
model.save('myCNNmodel.h5')